<img src="https://www.hennig-olsen.no/wp-content/uploads/2019/03/hennig-olsen-is.png" width="400" height="400">

# Optimal batch-production of ice cream
(_This is a fictive example. I have no relations to Hennig-Olsen except for being a passionate consumer of their products_)

On days production at Henning-Olsen consists of making their famous "Krone-is Sjokolade". This year, however, they have released a new and grander version of this chocolate ice cream, the "Krone-is Sjokolade XL", which is bigger in both size and price. When creating a batch, they have to follow the recipe and obey the available resources, while simultanously maximizing profit.

The recipes are as follows:
- **Original**: 1.0 dl cream, 25g sugar, 100g chocolate
- **Xl:** 1.3 dl cream, 40g sugar, 200g chocolate

In the storage, Henning-Olsen has the following amount of resources:
- **Cream:** 750 dl
- **Sugar:** 125 000 g
- **Chocolate:**: 10 000 g

Each ingredients has a cost:
- **Cream:** 6.12 kr per dl
- **Sugar:** 0.017 kr per gram
- **Chocolate:** 0.115 kr per gram

The ice creams are sold for the following price:
- **Original:** 23,-
- **XL:** 34,-

Requirements to production:
- **Original:** >500
- **XL:** >500


How much of each ice cream type should Hennig-Olsen produce to maximize their profit, while still obeying the included constaints?

# Formalizing the problem

## Summarize ingredients information
First, let's put the ingredients and resources into a table that better summarizes the available information

| Ice cream   | Cream [dl]  | Sugar [g] | Chocolate [g] |
| ----------- | ----------- | --------- | ------------- |
| Original    |         1.0 |       100 |            25 |
| XL          |         1.3 |       200 |            40 |
||
| Avail. res. |         750 |    125000 |       10000 | 

## Calculate profit per ice cream
Second, let us calculate the profit for each type of ice cream

In [1]:
org_kr = 21
xl_kr  = 36

cream_kr = 6.12
sugar_kr = 0.017
choc_kr  = 0.115

profit_org = org_kr - (1.0*cream_kr + 25*sugar_kr + 100*choc_kr)
profit_xl  = xl_kr  - (1.2*cream_kr + 40*sugar_kr + 200*choc_kr)

print(f"Profit per original: ~kr {profit_org:.2f}")
print(f"Profit per XL      : ~kr {profit_xl:.2f}")

Profit per original: ~kr 2.95
Profit per XL      : ~kr 4.98


## Create expression for the total profit
We wish to maximize the profit, where the profit for each ice cream was calculated above. We can use create an expression for the total profit for a given number of **original** and **XL** ice creams.

$$2.95 \cdot \textrm{original} + 4.98 \cdot \textrm{XL}$$

## Create expressions for the resource and minimum production rules
From the table above, we know that the following constaints needs to be followed

$$
\begin{align*}
\textbf{Cream: }     && 1.0 \textrm{ original} &+ 1.3 \textrm{ XL} &&\leq 750 \\
\textbf{Sugar: }     && 100 \textrm{ original} &+ 200 \textrm{ XL} &&\leq 125000 \\
\textbf{Chocolate: } &&  25 \textrm{ original} &+  40 \textrm{ XL} &&\leq 10000 \\
\end{align*}
$$

There are also some minimum specifications to the orders of each ice cream
$$
\begin{align*}
\textrm{original} & \geq 500 \\
\textrm{XL: } & \geq 500 \\
\end{align*}
$$

# Summarize the problem as a classical LP problem
$$
\begin{align*}
\textbf{Maximize} & \\
& 2.95 \cdot \textrm{original} + 4.98 \cdot \textrm{XL} \\
\textbf{Subject to:} & \\
& 1.0 \textrm{ original} + 1.3 \textrm{ XL} &&\leq 750 \\
& 100 \textrm{ original} + 200 \textrm{ XL} &&\leq 125000 \\
& 25 \textrm{ original} +  40 \textrm{ XL} &&\leq 10000 \\
& \textrm{original} &&\geq 500 \\
& \textrm{XL} &&\geq 500 \\
\end{align*}
$$


# Model the optimization problem
We will use the framework PuLP to model the problem above

In [2]:
import pulp

## Create a blank problem
In PuLP, we first create a blank problem, and then append the objective function and constraints to it later (almost like a list). Notice that we specify that this is a maximization problem. In PuLP, the default is a minimization problem.

In [52]:
prob = pulp.LpProblem(name="Maximize ice cream profit", sense=pulp.LpMaximize)

## Define constants and variables
The constants of the problem are the amount of ingredients and the amount of available resources. The variables of the problem are `original` and `xl`, which represents the number of ice creams to produce of each type.

In [53]:
# Recipe
cream_org = 1.0
sugar_org = 100
choco_org =  25

cream_xl  = 1.3
sugar_xl  = 200
choco_xl  =  40

# Available resources
max_cream =   2500
max_sugar = 300500
max_choco =  70000

# Minimum and maximum production
min_org = 500
min_xl = 500
max_tot = 2000

In [54]:
original = pulp.LpVariable(name="original", lowBound=0)
xl       = pulp.LpVariable(name="xl",       lowBound=0)

## Define and append the objective function to the problem
We can now append the objective function
$$
\begin{align*}
& 2.95 \cdot \textrm{original} + 4.98 \cdot \textrm{XL} \\
\end{align*}
$$

The constants above are limited to two decimals. For better accuracy, we implement the objective function using the profits that we calculated earlier.

In [55]:
prob += profit_org * original + profit_xl * xl

## Define and append the constraints
$$
\begin{align*}
& 1.0 \textrm{ original} + 1.3 \textrm{ XL} &&\leq 75000 \\
& 100 \textrm{ original} + 200 \textrm{ XL} &&\leq 325000 \\
& 25 \textrm{ original} +  40 \textrm{ XL} &&\leq 100000 \\
& \textrm{original} &&\geq 500 \\
& \textrm{XL} &&\geq 500 \\
\end{align*}
$$

In [56]:
prob += cream_org * original + cream_xl * xl <= max_cream, "cream"
prob += sugar_org * original + sugar_xl * xl <= max_sugar, "sugar"
prob += choco_org * original + choco_xl * xl <= max_choco, "chocolate"
prob += original >= min_org, "original min. limit"
prob += xl >= min_xl,  "xl min. limit"
prob += original + xl <= max_tot

## Verify formulation

In [57]:
print(prob)

Maximize ice cream profit:
MAXIMIZE
2.9549999999999983*original + 4.975999999999999*xl + 0.0
SUBJECT TO
cream: original + 1.3 xl <= 2500

sugar: 100 original + 200 xl <= 300500

chocolate: 25 original + 40 xl <= 70000

original_min._limit: original >= 500

xl_min._limit: xl >= 500

_C1: original + xl <= 2000

VARIABLES
original Continuous
xl Continuous



## Solve the LP maximization problem

In [60]:
status = prob.solve()

print("Solution status:", pulp.LpStatus[status].upper())
print()
print(f"Total profit:        {prob.objective.value():10,.2f} kr")
print()
print(f"num of ORIGINAL:     {original.value():>10.2f} pcs")
print(f"num of XL:           {xl.value():>10.2f} pcs")
print()
print(f"Remaining cream:     {max_cream - (1.0 * original.value() + 1.3 * xl.value()):>10.2f} dl")
print(f"Remaining sugar:     {max_sugar - (100 * original.value() + 200 * xl.value()):>10.2f} g")
print(f"Remaining chocolate: {max_choco -  (25 * original.value() +  40 * xl.value()):>10.2f} g")

Solution status: OPTIMAL

Total profit:          7,941.10 kr

num of ORIGINAL:         995.00 pcs
num of XL:              1005.00 pcs

Remaining cream:         198.50 dl
Remaining sugar:           0.00 g
Remaining chocolate:    4925.00 g
